## Inverse Cooking: Recipe Generation from Food Images

In [181]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time

Set ```data_dir``` to the path including vocabularies and model checkpoint

In [182]:
data_dir = ''

In [183]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'


# device = torch.device('cpu')
# map_loc = None


In [184]:
print(device)

cuda


In [185]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))

ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [186]:
def get_ingrs(ids):
    gen_ingrs = []
    for ingr_idx in ids:
        ingr_name = ingrs_vocab[ingr_idx]
        if ingr_name == '<pad>':
            break
        if ingr_name == '<end>':
            break
        gen_ingrs.append(ingr_name)
    return gen_ingrs


def mask_from_eos(ids, eos_value, mult_before=True):
    mask = torch.ones(ids.size()).to(device).byte()
    mask_aux = torch.ones(ids.size(0)).to(device).byte()

    # find eos in ingredient prediction
    for idx in range(ids.size(1)):
        # force mask to have 1s in the first position to avoid division by 0 when predictions start with eos
        if idx == 0:
            continue
        if mult_before:
            mask[:, idx] = mask[:, idx] * mask_aux
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
        else:
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
            mask[:, idx] = mask[:, idx] * mask_aux
    return mask
print (instrs_vocab_size, ingr_vocab_size)

23231 1488


In [34]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False
model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join('/home/ct2020dl5787/inversecooking/model/checkpoints', 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False
model.reduction = 'none'
print ('loaded model')
print ("Elapsed time:", time.time() -t)


loaded model
Elapsed time: 2.504643201828003


Set ```use_urls = True``` to get recipes for images in ```demo_urls```. 

You can also set ```use_urls = False``` and get recipes for images in the path in ```data_dir/test_imgs```.

In [144]:
import requests
from io import BytesIO
import random
from collections import Counter
unpickled_df = pd.read_pickle("df_recipe1023.pkl")
use_urls = True # set to true to load images from demo_urls instead of those in test_imgs folder
show_anyways = False #if True, it will show the recipe even if it's not valid
image_folder = os.path.join('/home/ct2020dl5787/inversecooking/data/'+'demo_imgs')
#TODO1:load data
# if not use_urls:
#     demo_imgs = os.listdir(image_folder)
#     random.shuffle(demo_imgs)

# demo_urls = ['https://food.fnr.sndimg.com/content/dam/images/food/fullset/2013/12/9/0/FNK_Cheesecake_s4x3.jpg.rend.hgtvcom.826.620.suffix/1387411272847.jpeg',
#          'https://www.196flavors.com/wp-content/uploads/2014/10/california-roll-3-FP.jpg']
demo_urls = unpickled_df["url"].tolist()
demo_files = demo_urls if use_urls else demo_imgs
# demo_files = demo_files

In [180]:
##STEP1: run image
err_pic = []
img_file_list = []
start = time.time()
for img_file in demo_files:
    
#    if use_urls:
    response = requests.get(img_file)
    try:
        image = Image.open(BytesIO(response.content))
    
        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)
        
        plt.imshow(image_transf)
        plt.axis('off')
        plt.show()
        plt.close()
        
        num_valid = 1
        img_file_list.append(img_file)
        if img_file == demo_files[0]:
            print(img_file)
            image_all = image_tensor
        else:
             image_all = torch.cat((image_all,image_tensor))
    except:
        print('error image')
        pass


In [176]:
##step2: run ingredient_decoder  
## batch_size could be changed

outputs = dict()
recipt_all = []
batch_size = 100


for i in range(image_all.shape[0]//batch_size + 1):

    img_features = model.image_encoder(image_all[i*batch_size : (i+1)*batch_size])
    ingr_ids, ingr_probs = model.ingredient_decoder.sample(None,None,greedy = True,temperature=1.0,
                                                           img_features=img_features, first_token_value=0,replacement=False)
    
    sample_mask = mask_from_eos(ingr_ids, eos_value=0, mult_before=False)
    ingr_ids[sample_mask == 0] = 0

    outputs['ingr_ids'] = ingr_ids
    outputs['ingr_probs'] = ingr_probs.data

    ingr_ids = outputs['ingr_ids'].cpu().numpy()
    recipe = list(map(lambda x : get_ingrs(x), ingr_ids))
        
    recipt_all.extend(recipe)

In [179]:
##step3:save info
generate_data = pd.DataFrame([img_file_list,recipt_all]).T
generate_data = generate_data.rename(columns={0: "url", 1: "generate_ingre"})
all_info = pd.merge(unpickled_df, generate_data , how='inner', on=['url'])
all_info.to_pickle('output/recipt_only.pkl')
all_info.head()

,id,ingredients,instructions,url,generate_ingre
0,00003a70b1,"[2 12 cups milk, 1 12 cups water, 14 cup butte...","[Preheat oven to 350 degrees Fahrenheit., Spra...","http://img.sndimg.com/food/image/upload/w_512,...","[cheese, onion, pepper, potato, milk, salt, bu..."
1,000075604a,"[2 Chicken thighs, 2 tsp Kombu tea, 1 White pe...",[Pierce the skin of the chicken with a fork or...,https://img-global.cpcdn.com/001_recipes/58069...,"[chicken, pepper, salt, lemon, oil, paprika, j..."
2,00007bfd16,"[6 -8 cups fresh rhubarb, or, 6 -8 cups frozen...",[Put ingredients in a buttered 9 x 12 x 2-inch...,"http://img.sndimg.com/food/image/upload/w_512,...","[sugar, egg, flour, rhubarb, salt, butter, cin..."
3,000095fc1d,"[8 ounces, weight Light Fat Free Vanilla Yogur...",[Layer all ingredients in a serving dish.],http://tastykitchen.com/recipes/wp-content/upl...,"[strawberries, yogurt, blueberries, banana, al..."
4,0000b1e2b5,"[1 teaspoon fennel seeds, 1 pound pork tenderl...","[Preheat oven to 350F with rack in middle., Cr...",http://assets.epicurious.com/photos/5609a4d662...,"[pepper, oil, salt, clove, onion, parsley]"


In [ ]:
##step4:evalutation
def get_ingred_f1(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        F-1 score of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.4
    '''
    intersection = list(set(pred) & set(label))
    precision = len(intersection) / len(pred)
    recall = len(intersection) / len(label)
    f1 = 2 * precision * recall / (precision + recall)
    return f1


def get_ingred_IOU(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        IOU of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.25
    '''
    intersection = len(list(set(pred) & set(label)))
    union = len(list(set(pred) | set(label)))
    iou = intersection / union
    return iou


import nltk
import string
def get_bleu_n_score(pred, label, n = 4):
    
    '''
    TODO: STEM not added
    input: 
        pred: One string of predict recipe 
        label: One string of reference recipe
        n(optional): up to n-gram.
    output: 
        bleu score
    
    i.e.  
    in: 
        pred = "Add the buttter"
        label = "Add half butter and mix well"
    out: 
        0.25
    '''
    weights = [1/n] * n
    pred_list = pred.translate(str.maketrans('', '', string.punctuation)).split()
    label_list = label.translate(str.maketrans('', '', string.punctuation)).split()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([label_list], pred_list, weights)
    return BLEUscore



from nltk.translate.meteor_score import meteor_score
def get_meteor_score(pred, label):
    return meteor_score([label], pred)